In [1]:
# necessary libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
# import data from csv file
df = pd.read_csv("flight_data_with_dist_and_dur.csv")

# count how often a departure airport is used and how often a destination airport is used
df1 = df.groupby(["DEPSTN"])["DEPSTN"].count()
df2 = df.groupby(["ARRSTN"])["ARRSTN"].count()

# bring both datasets together and prep for merge with datafram containing lat and lon data
df3 = pd.concat([df1, df2], axis=1)
df3 = df3.fillna(0)
df3["sum"] = df3["DEPSTN"] + df3["ARRSTN"]
df3 = df3.drop(["DEPSTN", "ARRSTN"], axis=1)
df3["ARRSTN"] = df3.index

# import iata data from csv file (source: https://github.com/ip2location/ip2location-iata-icao)
iata_data = pd.read_csv("iata-icao.csv")
iata_data = iata_data.drop(["country_code", "region_name", "icao", "airport"], axis=1)

# create dataframe to merge data
iata_data_arr = iata_data.rename(columns={"iata": "ARRSTN", "latitude": "arr_lat", "longitude": "arr_lon"})

# merge iata data with airport counts dataframe
df3 = pd.merge(df3, iata_data_arr, on="ARRSTN", how="left")

# make plot with dot size dependent on frequency of use
fig = px.scatter_map(df3, 
                        lat="arr_lat", 
                        lon="arr_lon", 
                        #color="sum",
                        size="sum",
                        zoom=2, 
                        height=900,
                        width=1100)

fig.update_traces(marker=dict(color="blue"),
                  selector=dict(mode='markers'))

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [3]:
# make plot with used airports without any additional condition
fig = px.scatter_map(df3, 
                        lat="arr_lat", 
                        lon="arr_lon", 
                        #color="sum",
                        #size="sum",
                        zoom=2, 
                        height=900,
                        width=1100)

fig.update_traces(marker=dict(color="blue", size=9),
                  selector=dict(mode='markers'))

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()